# Junior Notebook Hyperopt

Run and Model of Hyperparameter Training

In [1]:
# %load curriculumagent/junior/hyper_parameter_search_junior.py
"""
In this file, the code for the hyperparameter search of the junior model is provided.

We first set the range of the parameters and thereafter define the tune experiment
"""
import json
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from pathlib import Path
import os
from typing import Optional
import tensorflow as tf

from curriculumagent.junior.junior_student import Junior
from curriculumagent.junior.junior_student import load_dataset
import matplotlib.pyplot as plt
from lightsim2grid import LightSimBackend
import grid2op

example_path = os.getcwd()

In [2]:

tf.__version__
backend = LightSimBackend()
env_path = test_data_path = Path(example_path)/ "data" / "training_data_track1"
print(env_path)
env = grid2op.make(env_path, backend=backend)
env.set_id(1)
env.reset()
env.chronics_handler.get_name()
obs = env.get_obs()

In [3]:
best_params_path = Path(example_path) /"data"/ "junior_best_params.json"

with open(best_params_path) as json_file:
    best_params = json.load(json_file)

best_params["epochs"] = 1000
for name in ["layer1","layer2","layer3","layer4"]:
        best_params[name] = np.round(best_params[name])
    
best_params

## Now run final training: 

In [4]:
path_to_files = Path(example_path) / "data" / "junior_experience"
s_train, a_train, s_validate, a_validate, _, _ = load_dataset(path_to_files, dataset_name="junior_dataset")

scaler = MinMaxScaler()#StandardScaler()
s_tr_t = scaler.fit_transform(s_train)
s_val_t = scaler.transform(s_validate)

test_data_path = Path(example_path) / "data"
path1 = test_data_path / "actionspace_nminus1.npy"
path2 = test_data_path / "actionspace_tuples.npy"
action_set = [path1, path2]
    
# Save the scaler: 
with open( Path(example_path) / "data" /"scaler_junior.pkl", "wb") as fp:   #Pickling
    pickle.dump(scaler, fp)

junior = Junior(config=best_params,
                action_space_file=action_set, seed = 42)

history = junior.train(log_dir= Path(example_path) / "data" / "junior_hyperopt" / "train",
                       ckpt_dir= Path(example_path) / "data" / "junior_hyperopt" / "train",
                       patience=50,
                       x_train=s_tr_t, y_train=a_train,
                       x_validate=s_val_t, y_validate=a_validate)
print(junior.model.summary())
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])

junior.model.save(Path(example_path) / "data" / "junior_hyperopt" / "junior_hyperopt_model")
